(Example from the Book p.29 ff)

A lighthouse is somewhere off a piece of straight coastline at a position α (alpha) along the shore and a distance <br>
β (beta) out at sea. It emits a series of short highly collimated flashes at random intervals and hence at random azimuths. <br>
These pulses are intercepted on the coast by photo-detectors that record only the fact that a flash has occurred, but not <br>
the angle from which it came. <br>
N flashes have so far been recorded at positions {xk}. Where is the lighthouse?’

<img src="pictures01/lightHouseGeometry.jpg">

In [9]:
# generate data to analyze:
import numpy as np

# position of light-house on x-axis:
alpha = 1

# position of light-house off shore:
beta = 4

def data_from_angle(thetas, alpha, beta):
    return beta * np.tan(thetas) + alpha

def generate_data(numberOfDataPoints = 10):
    return np.random.uniform(size=numberOfDataPoints, low=-np.pi/2, high=np.pi/2)

# test functions:
theta_ks = generate_data()
print(theta_ks)

x_ks = data_from_angle(thetas=theta_ks, alpha=alpha, beta=beta)
print(x_ks)


[-1.18027881 -0.33207798  0.64961123 -1.4043952  -0.73957294  0.56570059
  1.4381734   0.9824136   0.84808338 -0.67753555]
[ -8.71675638  -0.3793932    4.03836457 -22.81602052  -2.64922689
   3.53967762  30.98365863   6.99506125   5.53576839  -2.21837373]


If we assume a uniform distribution, $ prob(\theta | \{x_k\}, \alpha, \beta, I) = \frac{1}{\pi}$ for $\theta$ between $[- \pi/2 , + \pi/2]$, we have a Cauchy-Distribution vor the $x_k$ :

$$
prob(x_k | \alpha, \beta, I) = \frac{\beta}{\pi [\beta^2 + (x_k - \alpha)^2]}
$$

Using Bayes' theorem and assuming independence for the measurements, the log-prob function L is given by:

$$
  L = log(prob(  \alpha, \beta | \{x_k\} , I) = const + \sum^N_{k=1}{(log(\beta) - log(\beta^2 + (x_k - \alpha)^2))}
$$

To find our best estimate for $\alpha$ and $\beta$ we take partial derivatives equal zero and thus end up with a non-liniear optimization problem / non-linear root finding problem that we will have to solve numerically:

$$
\frac{\partial L}{\partial \alpha} = \sum^N_{k=1}{\frac{2 (x_k - \alpha)}{\beta^2 + (x_k - \alpha)^2}} = 0 \\


\frac{\partial L}{\partial \beta} = \frac{N}{\beta} - \sum^N_{k=1}{\frac{2 \beta}{\beta^2 + (x_k - \alpha)^2}} = 0
$$

In [1]:
import numpy as np

def gradient_descent(gradient_f, start, learn_rate, n_iter=50, tolerance=1e-06):
    vector = start
    for _ in range(n_iter):
        diff = -learn_rate * gradient_f(vector)
        if np.all(np.abs(diff) <= tolerance):
            break
        vector += diff
    return vector

# test of function for f(x)=X**2:

gradient_descent(gradient_f=lambda x: 2*x, start=10, learn_rate=0.2)


2.210739197207331e-06

In [6]:
# test of function for f(x, y)= ( x**2 , y**2 + 1):

def grad_f(x):
    return np.array([2*x[0], 2*x[1]])

gradient_descent(gradient_f=grad_f, start=np.array([10.0, 10.0]), learn_rate=0.2)


array([2.2107392e-06, 2.2107392e-06])

In [ ]:
# TO DO: use numpy operations everywhere:
def dL_da(start, x_ks):
    x_minus_a = x_ks - start[0]
    numerator = 2 * x_minus_a
    denominator = start[1]**2 + x_minus_a**2
    fractions = numerator/denominator
    return np.sum(fractions)  


In [ ]:
# TO DO: use numpy operations everywhere:
def dL_db(start, x_ks):
    N = np.size(x_ks)
    x_minus_a = x_ks - start[0]
    numerator = 2 * start[1]
    denominator = start[1]**2 + x_minus_a**2
    fractions = numerator/denominator
    return N/start[1] - np.sum(fractions)  

In [ ]:
def gradient_L(start, x_ks):
    return np.array([dL_da(start, x_ks), dL_db(start, x_ks)])